In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import os
import urllib.parse

In [2]:
def get_random_page_urls(driver, languages):
    urls = {}
    for language in languages:
        driver.get('https://{}.wikipedia.org/wiki/Special:Random'.format(language))
        urls[language] = driver.current_url
    return urls

Je commence par définir une fonction nommée `get_random_page_urls` qui prend deux paramètres : `driver` et `languages`. Le `driver` est probablement un objet qui me permet d'interagir avec un navigateur web, et `languages` est une liste des codes de langue pour lesquels je veux récupérer des URL aléatoires de pages Wikipédia.

À l'intérieur de la fonction, je crée un dictionnaire vide appelé `urls` où je vais stocker les URL que je récupère. Pour chaque langue présente dans la liste `languages`, je fais plusieurs actions :

1. Je construis l'URL de la page Wikipédia aléatoire pour cette langue en utilisant la méthode `format`. Cela me donne une URL comme 'https://en.wikipedia.org/wiki/Special:Random' si la langue est 'en' (anglais), par exemple.

2. J'utilise le `driver` pour charger cette URL dans le navigateur web avec `driver.get(url_formée)`.

3. Une fois que la page a chargé, je récupère l'URL actuelle, qui est une page spécifique choisie aléatoirement par Wikipédia, en utilisant `driver.current_url`.

4. Je stocke cette URL dans le dictionnaire `urls` avec la langue correspondante comme clé.

Après avoir traité toutes les langues dans la boucle, je retourne le dictionnaire `urls` qui contient les codes de langue comme clés et les URL des pages Wikipédia aléatoires comme valeurs.

In [3]:
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


def get_interlanguage_urls(driver, base_url):
    driver.get(base_url)
    wait = WebDriverWait(driver, 30)
    try:
        interlanguage_links = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.interlanguage-link a')))
        return {link.get_attribute('lang'): link.get_attribute('href') for link in interlanguage_links}
    except TimeoutException:
        logging.error(f"Timeout when trying to find interlanguage links for {base_url}")
        return {}

Je commence par importer le module `logging` qui me permet d'enregistrer des informations sur ce qui se passe dans mon code. Je configure le niveau de logging à `INFO`, ce qui signifie que je vais enregistrer les messages de niveau INFO et supérieur (comme ERROR). Je définis également un format spécifique pour ces messages qui inclut la date et l'heure, le niveau de gravité du message et le message lui-même.

Ensuite, je définis une fonction nommée `get_interlanguage_urls` qui prend deux paramètres : `driver` et `base_url`. Le `driver` est l'objet qui interagit avec le navigateur, et `base_url` est l'URL de la page de base à partir de laquelle je veux récupérer des liens interlangues.

À l'intérieur de la fonction, je fais plusieurs actions :

1. Je charge l'URL de base dans le navigateur en utilisant `driver.get(base_url)`.

2. Je crée un objet `wait` de type `WebDriverWait`, lié au `driver` et avec un délai maximal de 30 secondes. Cet objet me permet d'attendre que certaines conditions soient remplies avant de continuer l'exécution de mon code.

3. J'essaie ensuite de récupérer tous les éléments de la page qui correspondent à des liens interlangues. Ces liens sont localisés par un sélecteur CSS qui cible les éléments 'a' dans les 'li' avec la classe 'interlanguage-link'. J'attends que ces éléments soient présents sur la page, ce que je vérifie avec la méthode `presence_of_all_elements_located`.

4. Si les éléments sont trouvés à temps, je construis un dictionnaire avec les langues comme clés (en récupérant l'attribut 'lang' de chaque lien) et les URL correspondantes comme valeurs (en récupérant l'attribut 'href' de chaque lien).

5. Si le délai d'attente de 30 secondes est dépassé sans que les liens soient trouvés, je consigne une erreur de type `TimeoutException` avec un message d'erreur spécifique qui mentionne l'URL de base. Dans ce cas, je retourne un dictionnaire vide.

La fonction `get_interlanguage_urls` retourne donc soit un dictionnaire contenant les langues et leurs URL associées si tout se passe bien, soit un dictionnaire vide si une erreur se produit lors de la récupération des liens.

In [4]:
def extract_paragraphs(driver, url):
    driver.get(url)
    
    paragraphs = ""

    # Extract the first paragraph of the article
    try:
        title = os.path.basename(urllib.parse.urlparse(url).path)
        elements = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div/p')

        print(f"Found {len(elements)} paragraphs for {url} : {[el.text[:30] for el in elements]}")

        for el in elements:
            if el.text:
                paragraphs += el.text + "\n"
            else:
                continue

    except:
        print(f"Could not find paragraph for {url}")

    return paragraphs, title

Je définis ici une fonction appelée `extract_paragraphs` qui prend deux paramètres : `driver` et `url`.

Voici ce que je fais dans cette fonction :

1. Je commence par charger l'URL donnée dans le navigateur à l'aide de `driver.get(url)`.

2. Je déclare une variable `paragraphs` que j'initialise avec une chaîne de caractères vide. Cela servira à stocker tous les paragraphes de texte que je vais extraire.

3. Je tente ensuite d'extraire le premier paragraphe de l'article. Pour cela, je procède comme suit :

   a. J'extrais le titre de la page à partir de l'URL en utilisant les fonctions `os.path.basename` et `urllib.parse.urlparse` pour obtenir uniquement la partie finale de l'URL, qui correspond généralement au titre de l'article.

   b. Je recherche dans le document tous les éléments qui correspondent au chemin XPath spécifié. Ce chemin cible les paragraphes situés dans une division particulière de la page identifiée par l'identifiant `mw-content-text`.

   c. J'affiche un message indiquant combien de paragraphes j'ai trouvés pour l'URL donnée. Pour chaque élément trouvé, je montre les 30 premiers caractères du texte de cet élément.

   d. Je parcours ensuite chaque élément trouvé. Si l'élément contient du texte, je l'ajoute à la variable `paragraphs` avec un saut de ligne à la fin. Si l'élément ne contient pas de texte, je continue simplement sans l'ajouter.

4. Si une exception se produit pendant l'extraction (par exemple, si le chemin XPath est incorrect ou si la page ne se charge pas correctement), j'affiche un message d'erreur indiquant que je n'ai pas pu trouver de paragraphe pour l'URL donnée.

5. Enfin, je retourne un tuple contenant les paragraphes extraits et le titre de l'article. Cela permet à celui qui utilise cette fonction de recevoir à la fois le texte extrait et le titre de la page à partir de laquelle le texte a été extrait.

In [5]:
def write_txt(data_folder, lang, url, paragraph, file_counts):
    # Parse the URL to extract the article title, which is the part after the last '/'
    parsed_url = urllib.parse.urlparse(url)
    article_title = os.path.basename(parsed_url.path)
    
    # URL decode the article title (e.g., replace '%E1%83%90' with actual characters)
    article_title = urllib.parse.unquote(article_title)
    
    # Replace any remaining invalid file name characters (like '/') in the title with '_'
    valid_article_title = ''.join(char if char.isalnum() else '_' for char in article_title)
    
    # Ensure the data folder exists
    os.makedirs(data_folder, exist_ok=True)
    
    # Generate the filename and write the paragraph to the file
    filename = f"{lang}_{valid_article_title}.txt"
    file_path = os.path.join(data_folder, filename)
    
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(paragraph)
        print(f"Successfully wrote {file_path}\n")
    file_counts[lang] += 1


Je définis une fonction nommée `write_txt` qui prend cinq paramètres : `data_folder`, `lang`, `url`, `paragraph`, et `file_counts`. Ces paramètres me servent respectivement à spécifier où sauvegarder les fichiers, la langue de l'article, l'URL de l'article, le paragraphe à écrire, et un dictionnaire pour suivre combien de fichiers j'ai écrits pour chaque langue.

Voici comment je procède dans cette fonction :

1. **Extraction du titre de l'article depuis l'URL** : 
   - Je parse l'URL pour isoler la partie qui me sert de titre, qui est après le dernier '/' dans le chemin de l'URL.
   - Je décode le titre pour convertir des séquences d'échappement URL (comme '%E1%83%90') en caractères lisibles.

2. **Nettoyage du titre** :
   - Je remplace tout caractère du titre qui n'est pas alphanumérique par un underscore ('_') pour m'assurer que le titre peut être utilisé comme nom de fichier sans causer d'erreur.

3. **Préparation du dossier de données** :
   - Je m'assure que le dossier où je veux sauvegarder les fichiers existe. S'il n'existe pas, je le crée avec `os.makedirs`, et l'option `exist_ok=True` me permet de ne pas générer d'erreur si le dossier existe déjà.

4. **Écriture du paragraphe dans un fichier** :
   - Je génère le nom du fichier en utilisant la langue et le titre de l'article nettoyé, et j'ajoute l'extension '.txt'.
   - Je combine le dossier de données et le nom du fichier pour créer le chemin complet du fichier.
   - J'ouvre le fichier en mode écriture (`'w'`) avec encodage UTF-8, et j'écris le paragraphe dedans. Je termine par afficher un message pour confirmer que l'écriture a réussi.

5. **Incrémentation du compteur de fichiers** :
   - J'augmente de 1 le compteur pour la langue correspondante dans le dictionnaire `file_counts` pour suivre combien de fichiers ont été écrits pour chaque langue.

La fonction `write_txt` n'a pas de valeur de retour. Elle effectue son travail d'écriture en sauvegardant les paragraphes dans des fichiers spécifiques tout en organisant et en comptant combien de fois elle le fait pour chaque langue.

In [6]:
def main():
    driver = webdriver.Chrome()
    data_folder = "../../../data/raw/"
    languages = ['ar', 'fr', 'en', 'de', 'es', 'ja', 'ko', 'ru', 'zh']
    file_counts = {lang: 0 for lang in languages}

    base_urls = get_random_page_urls(driver, languages)
    for lang in languages:
        while file_counts[lang] < 20:
            current_url = base_urls[lang]
            paragraphs, article_title = extract_paragraphs(driver, current_url)
            if paragraphs:
                write_txt(data_folder, lang, article_title, paragraphs, file_counts)
            
            links = get_interlanguage_urls(driver, current_url)
            for link_lang, link_url in links.items():
                if link_lang in languages and file_counts[link_lang] < 20:
                    link_paragraphs, title = extract_paragraphs(driver, link_url)
                    if link_paragraphs:
                        write_txt(data_folder, link_lang, title, link_paragraphs, file_counts)

            # Refreash the base URL for the current language
            new_urls = get_random_page_urls(driver, [lang])
            base_urls[lang] = new_urls[lang]
            
    driver.quit()



J'ai défini une fonction nommée `main` qui orchestre l'extraction et l'enregistrement de textes à partir de différentes éditions linguistiques de Wikipedia. Voici comment je m'y prends :

1. **Initialisation du pilote du navigateur** :
   - J'initialise un pilote Chrome en utilisant `webdriver.Chrome()`. Cela me permet d'ouvrir et de manipuler un navigateur Chrome directement depuis mon script.

2. **Définition du dossier de données et des langues** :
   - Je définis le chemin vers le dossier où je souhaite enregistrer les fichiers extraits. Je spécifie également une liste de codes de langue pour les éditions de Wikipedia que je souhaite explorer.

3. **Initialisation des compteurs de fichiers** :
   - Je crée un dictionnaire `file_counts` pour suivre le nombre de fichiers que j'ai écrits pour chaque langue, en initialisant tous les compteurs à zéro.

4. **Récupération des URL initiales** :
   - J'utilise la fonction `get_random_page_urls` pour obtenir une URL de page aléatoire pour chaque langue listée. Cela me donne un point de départ pour chaque langue.

5. **Traitement pour chaque langue** :
   - Pour chaque langue, je continue à extraire et enregistrer des paragraphes tant que je n'ai pas atteint 20 fichiers pour cette langue. Je récupère l'URL actuelle pour la langue en question et j'extrait des paragraphes et le titre de l'article à partir de cette URL.

6. **Écriture des paragraphes** :
   - Si des paragraphes sont extraits, je les enregistre en utilisant la fonction `write_txt`.

7. **Exploration des liens interlangues** :
   - J'utilise `get_interlanguage_urls` pour récupérer des liens vers d'autres éditions linguistiques de la même page Wikipedia. Pour chaque lien valide qui pointe vers une langue que je suis en train de traiter et où je n'ai pas encore atteint la limite de 20 fichiers, j'extrait et j'enregistre les paragraphes.

8. **Rafraîchissement des URL** :
   - Après avoir traité les liens interlangues, je rafraîchis l'URL de base pour la langue actuelle en récupérant une nouvelle URL aléatoire. Cela me permet de continuer à extraire de nouveaux contenus sans boucler sur les mêmes pages.

9. **Nettoyage** :
   - Une fois que j'ai fini de traiter toutes les langues, je ferme le pilote du navigateur avec `driver.quit()` pour libérer les ressources.

Cette fonction `main` organise donc systématiquement l'extraction de contenus multilingues à partir de Wikipedia, les sauvegarde localement, et assure que le nombre de fichiers pour chaque langue atteint un seuil spécifié. Cela me permet de construire une collection diversifiée de textes pour différents usages, comme l'entraînement de modèles de traitement de langues.

In [7]:
main()

Found 1 paragraphs for https://ar.wikipedia.org/wiki/%D8%AF%D9%88%D8%B1%D9%8A_%D8%A7%D9%84%D8%A8%D9%88%D8%B3%D9%86%D8%A9_%D9%88%D8%A7%D9%84%D9%87%D8%B1%D8%B3%D9%83_%D8%A7%D9%84%D9%85%D9%85%D8%AA%D8%A7%D8%B2_2002%E2%80%9303 : ['دوري البوسنة والهرسك الممتاز 2']

Successfully wrote ../../../data/raw/ar_دوري_البوسنة_والهرسك_الممتاز_2002_03.txt
Found 6 paragraphs for https://de.wikipedia.org/wiki/Premijer_Liga_2002/03 : ['Die Premijer Liga 2002/03, die', 'Mit dieser Spielzeit, in der z', 'In dieser Saison mussten insge', 'Außerdem wurde der Ligabetrieb', 'Die 20 Mannschaften traten an ', 'Platzierungskriterien: 1. Punk']

Successfully wrote ../../../data/raw/de_03.txt
Found 3 paragraphs for https://en.wikipedia.org/wiki/2002%E2%80%9303_Premier_League_of_Bosnia_and_Herzegovina : ['The 2002–03 Premier League of ', 'The league was won by FK Leota', 'FK Leotar Trebinje (Coach: - M']

Successfully wrote ../../../data/raw/en_2002_03_Premier_League_of_Bosnia_and_Herzegovina.txt
Found 5 paragraphs 

2024-04-23 21:04:36,472 - ERROR - Timeout when trying to find interlanguage links for https://ar.wikipedia.org/wiki/%D9%85%D8%B4_%D8%A3%D9%84%D9%81_%D9%84%D9%8A%D9%84%D8%A9_%D9%88%D9%84%D9%8A%D9%84%D8%A9_(%D9%85%D8%B3%D9%84%D8%B3%D9%84)


Found 4 paragraphs for https://ar.wikipedia.org/wiki/%D8%A3%D9%86%D8%AA%D9%88%D9%86%D9%8A_%D9%81%D8%B1%D9%86%D8%A7%D9%86%D8%AF%D9%8A%D8%B2_(%D8%B3%D9%8A%D8%A7%D8%B3%D9%8A) : ['أنتوني فرنانديز (بالكتالونية: ', 'تولى منصب رئيس مجلس الإدارة، F', '.', '']

Successfully wrote ../../../data/raw/ar_أنتوني_فرنانديز__سياسي_.txt
Found 9 paragraphs for https://es.wikipedia.org/wiki/Antoni_Fern%C3%A1ndez_Teixid%C3%B3 : ['Antoni Fernández Teixidó (Barc', 'Se licenció en Ciencias Empres', 'Inicia su vida política en la ', 'En julio de 1991, Adolfo Suáre', 'Sin embargo, el enfrentamiento', 'En 1993 ingresó en Convergènci', 'En septiembre de 2006 el diari', 'Es uno de los principales expo', 'En 2017 participó en la formac']

Successfully wrote ../../../data/raw/es_Antoni_Fernández_Teixidó.txt
Found 1 paragraphs for https://ar.wikipedia.org/wiki/786 : ['سنة 786 م (بالأرقام الرومانية:']

Successfully wrote ../../../data/raw/ar_786.txt
Found 0 paragraphs for https://de.wikipedia.org/wiki/786 : []
Found 3

2024-04-23 21:05:29,695 - ERROR - Timeout when trying to find interlanguage links for https://ar.wikipedia.org/wiki/%D9%8A%D9%88%D8%B3%D9%81_%D8%A7%D9%84%D9%85%D8%A7%D9%84%D9%83%D9%8A


Found 2 paragraphs for https://ar.wikipedia.org/wiki/%D9%84%D9%8A%D9%88_%D8%B4%D9%8A%D8%B4%D9%8A : ['ليو شيشي هي ممثلة صينية ولدت ف', '']

Successfully wrote ../../../data/raw/ar_ليو_شيشي.txt
Found 19 paragraphs for https://en.wikipedia.org/wiki/Liu_Shishi : ['Liu Shishi (Chinese: 刘诗诗; piny', 'Professionally trained in ball', 'In 2007, Liu graduated from th', 'In 2009, Liu gained popularity', 'In 2010, Liu starred in the hi', "The year 2011 saw Liu's furthe", 'Liu then focused her career on', 'In 2012, Liu starred in action', 'In 2013, Liu starred in action', 'In 2014, Liu returned to the s', 'In 2016, Liu headlined the his', 'In 2017, Liu starred along vet', 'In 2019, Liu starred in the ro', "Liu Shishi's grandfather, Liu ", "Both of Liu's parents had been", 'Liu married Scarlet Heart co-s', 'On April 27, 2019, Liu gave bi', '', '2020|Cosmo Glam Night |Person ']

Successfully wrote ../../../data/raw/en_Liu_Shishi.txt
Found 10 paragraphs for https://es.wikipedia.org/wiki/Liu_Shishi : [

...

Found 3 paragraphs for https://ko.wikipedia.org/wiki/%EB%B7%98%EC%8A%A4%EC%97%94%ED%84%B0%ED%85%8C%EC%9D%B8%EB%A8%BC%ED%8A%B8 : ['', '뷘스엔터테인먼트(vvuins entertainment)', '본사는 2017년 6월 대구에서 주식회사로 설립되었으며']

Successfully wrote ../../../data/raw/ko_뷘스엔터테인먼트.txt


2024-04-23 21:15:59,345 - ERROR - Timeout when trying to find interlanguage links for https://ko.wikipedia.org/wiki/%EB%B7%98%EC%8A%A4%EC%97%94%ED%84%B0%ED%85%8C%EC%9D%B8%EB%A8%BC%ED%8A%B8


Found 4 paragraphs for https://ko.wikipedia.org/wiki/%EC%9C%A0%EA%B2%BD_(%EC%A0%84%ED%95%9C) : ['유경(劉敬, ? ~ ?)은 중국 전한 초기의 정치가다.', '제나라 사람으로, 고제 5년(기원전 202년) 농서에 ', '고제 7년(기원전 200년), 한왕 신이 흉노와 결탁해', '고제가 이후 흉노에 대한 대책을 묻자, 싸울 수도 없고']

Successfully wrote ../../../data/raw/ko_유경__전한_.txt
Found 5 paragraphs for https://zh.wikipedia.org/wiki/%E5%8A%89%E6%95%AC_(%E8%A5%BF%E6%BC%A2) : ['劉敬（?—?），本名婁敬，西漢初齊國盧邑（今山東省济南長清）', '婁敬本是齊國的戍卒，正被發往隴西（今甘肅一帶）戍邊，同鄉虞將', '漢高祖七年（前200年），劉邦派十餘批使節出使匈奴，匈奴故意', '劉邦先到平城（今山西省大同市），主力未至，冒頓单于傾全國40', '[在维基数据编辑]']

Successfully wrote ../../../data/raw/zh_劉敬__西漢_.txt
Found 5 paragraphs for https://ko.wikipedia.org/wiki/%EB%AA%BB%EA%B3%A8%EC%97%AD : ['', '못골역(Motgol station, 못골驛)은 부산광역', '부산 남구 대연동에 있던 옛 지명인 못골 마을에서 유래', '2면 2선 상대식 승강장이 있는 지하역이다. 스크린도어', '대합실을 통과해, 반대편 승강장으로 이동이 가능하다.']

Successfully wrote ../../../data/raw/ko_못골역.txt
Found 3 paragraphs for https://zh.wikipedia.org/wiki/%E6%B1%A0%E8%B0%B7%E7%AB%99_(%E9%9F%93%E5%9C%8B) : ['池谷站（朝鮮語：못골역／못골驛 Motgol yeok */', '站體為2面

2024-04-23 21:16:41,013 - ERROR - Timeout when trying to find interlanguage links for https://zh.wikipedia.org/wiki/%E7%94%84%E8%B4%9E


Found 7 paragraphs for https://zh.wikipedia.org/wiki/%E8%BF%AA%E9%87%8C_(%E7%B4%A2%E5%A7%86%E7%9C%81) : ['迪里（法語：Dury，法语发音：[dyʁi]）是法国上法兰西', '迪里（49°50\'43"N, 2°16\'10"E）面积10.', '与迪里接壤的市镇（或旧市镇、城区）包括：亚眠、埃贝库尔、圣菲', '迪里的时区为UTC+01:00、UTC+02:00（夏令时）', '迪里的邮政编码为80480，INSEE市镇编码为80261。', '迪里所属的省级选区为亚眠第六县（英语：Canton of A', '迪里于2021年1月1日时的人口数量为1455人。']

Successfully wrote ../../../data/raw/zh_迪里__索姆省_.txt
Found 5 paragraphs for https://zh.wikipedia.org/wiki/Area : ['Area是成立於1972年的義大利前衛搖滾與融合音樂的團體。', '在70年代的義大利前衛搖滾風潮中，Area是幾個有重要影響力', 'Area發行過四張錄音室專輯以及一張演唱會專輯，這是在Dem', 'Fariselli Project - Fariselli與', '進一步的資訊：http://www.italianprog.']

Successfully wrote ../../../data/raw/zh_Area.txt
